## Debug the SparsDT
import lib

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
import torch
import numpy as np
import lib.models as model
import lib.util as util

### Testing estimating alpha

In [11]:
Y = torch.rand([10,1600],dtype=torch.float32)
SparsDT = model.ModelSparseDT(10,Y, num_samples=1e5)

================= estimating alpha =================


### Testing the optimizer

In [12]:
args = {
    'lr' : 0.1
}
SparsDT.set_optimizer('SGD', args)

### Testing  estim gamma

In [13]:
U = torch.randn([SparsDT.m_rows, 1000])
a = SparsDT._estimate_gamma(U)
a.shape

torch.Size([1000])

### Testing find U, norm alpha, Loss

In [14]:
U = SparsDT._findU(1000)
U.shape

torch.Size([10, 1010])

In [15]:
norm = SparsDT._norm_alpha(SparsDT.A, U)
norm.shape

torch.Size([1010])

In [16]:
gamma = SparsDT._estimate_gamma(U)
l = SparsDT.Loss(U,gamma)
l

tensor(0.2697, grad_fn=<MeanBackward1>)

### Testing fit

In [17]:
A = SparsDT.fit(max_iter=1e3, rand_U=True, num_col = 100)

### Testing SparsDT on a SalphaS signal

In [36]:
m = 3 
n = 4
k = 1000
alpha = 1
beta = 0

In [37]:
A = util.Generate_A(m,n)
X = util.Generate_alpha_random(alpha, beta, shape=(n, k))
Y = A.mm(X)

In [38]:
SparsDT = model.ModelSparseDT(n,Y, num_samples=1e5)

================= estimating alpha =================


In [39]:
args = {
    'lr' : 1
}
SparsDT.set_optimizer('Adagrad', args)

In [40]:
Ahat = SparsDT.fit(max_iter=1000, rand_U=True)

In [35]:
util.calc_correct(Ahat, A)

tensor([0.9999, 0.9999, 0.9999, 1.0000])